# How to detect subcellular Ca<sup>2+</sup> signals using Flika
by Kyle Ellefsen, October 31, 2015


This tutorial is relevant for researchers who record subcellular Ca<sup>2+</sup> signals using fluorescent Ca<sup>2+</sup> indicators.  This algorithm automates the detection and analysis of these brief Ca2+ signals.  This is not a tutorial for researchers looking to analyze global Ca<sup>2+</sup> signals, although I'm planning on creating a tutorial for that, and many of the same concepts apply.

In this problem, we will use the threshold-cluster algorithm.  This algorithm is not published, but is modified from the algorithm published in [Ellefsen et al, 2014](http://dx.doi.org/10.1016/j.ceca.2014.06.003).  When using this algorithm, please cite that paper.  The threshold-cluster algorithm can be conceptually divided into two steps.  The first step preprocesses the raw movie and creates three movies.  The second step performs analysis on those movies according to user defined parameters.  I'll break this tutorial into those two steps.  

Using this algorithm involves reading and writing code.  Although this algorithm can all be done using Flika's GUI tools (opening windows, clicking buttons, etc.), it is much more efficient for a researcher to write a python file that will automatically perform all these operations.  Therefore, in this tutorial, I will be describing the code you'll need to use in each step.  If you do not know how to code in python, I recommend downloading [anaconda](https://www.continuum.io/downloads), which contains python and many python packages.  Then launch [Spyder](https://github.com/spyder-ide/spyder) (which comes with anaconda), and from within Spyder, run [Flika.py](https://github.com/kyleellefsen/Flika).  All these commands will be executed in the same console that Flika.py is running in. If you have questions about how to do that, email me and I'll help you with setting up.

## Preprocessing
There are three movies we want to generate as part of the preprocessing step.
1. $F/F_0$ Movie
2. Normalized Movie
3. Binary Movie

First, let's generate the $F/F_0$ movie. Each value of the $F/F_0$ movie represents a percent change from baseline fluoresences.  A value of 1 indicates that pixel is currently exactly at baseline fluoresence.  A value of 1.14 indicates a 14% increase above baseline fluorsence. To generate this movie, we will first open our file.

In [ ]:
open_file()

Then we will subtract the camera's black level.  The black level is the value of a pixel when 0 photons are detected.  There are two common ways to measure black level.  You could turn the laser off at the beginning of your record, and switch it on when you've recorded enough frames to accurately determine the black level.  Or you could measure the values in a black area of your field of view.  Let's say our black level is 120.

In [ ]:
subtract(720)

Operations in Flika are performed on the current window, which is either the window that was most recently created or the window that was last clicked on.  

Now, for each pixel, we want the percent change from baseline.  To calculate this, we need a measure of baseline fluorescence, during which there is no Ca<sup>2+</sup> activity.  Let's suppose that our cell is quiescent from frame 0 to frame 30.  Then we will take the average image from frame 0 to 30, and divide every image in the movie by that average image.  In Flika, this is performed in a single command:

In [ ]:
data_window=ratio(150,300,'average'); #ratio(first_frame, nFrames, ratio_type), now we are in F/F0
data_window.setName('Data Window dF/F0')

Notice that we set the variable 'data_window'.  The output of each Flika command returns the new window created by the command, which we can assign to a variable for later use.  We now have our data_window.  

The normalized movie should have several special properties that will enable automated signal detection.  The noise for each pixel should be gaussian distributed with a mean of 0 and a standard deviation of 1 ($\mu = 0; \sigma =1$).  Since data files won't start off with those properties, we need to transform them. 

Since we are concerned with only local signals, sometimes we want to remove global signals or other slow noise.  We can exploit the fact that global signals typically have a must slower timecourse than brief local signals, and use a temporal filter to remove slow changes.  We will use the butterworth filter.  In order to determine what parameters we want to use, let's use the the butterworth filter GUI.  Right click and drag on the $F/F_0$ movie window to create an ROI.  Right click in the center of that ROI, and plot the trace.  Then, in the main Flika window, select Process->Filters->Butterworth Filter.  Make sure the Preview checkbox is checked.  Adjust the Low Cutoff Frequency until the global signals are removed, but the local signals are still visible.  Then press the 'Cancel' button and use the parameters you found to perform the filter.  Sometimes this filter can take a few minutes.



In [ ]:
butterworth_filter(1,.03,1,keepSourceWindow=True)

If you don't need to filter your data, just subtract 1 from each value so that the noise is centered around 0.  

In [ ]:
subtract(1) #Don't do this if you used the filter above

Now our movie should have its noise centered around 0.  We want the standard deviation of our noise to be equal to 1, so we'll find a quiescent section of our movie and ratio by the standard deviation, similar to when we ratioed by the average.

In [ ]:
ratio(100,100,'standard deviation')

Our butterworth filter created artifacts at the beginning and end of the movie.  Let's eliminate those by setting the values at the beginning and end of the movie to 0. 'mt' is a variable that represents number of frames of our movie.

In [ ]:
norm_window=set_value(0,0,100) #to get rid of butterworth artifact at the beginning of the movie
norm_window=set_value(0,mt-150,mt-1) #to get rid of butterworth artifact at the end of the movie
norm_window.setName('Normalized Window')

Great! We're most of the way there.  We have our $F/F_0$ Movie and our Normalized Movie.  The last movie we need is the Binary Movie.  If we threshold our normalized window, so that all the pixels below a certain value are set to 0 and all pixels above that value are set to 1, the result would be a binary movie.  This is how I originally wrote the algorithm, and it will still work.  However, it is faster to gaussian blur every frame in an image and threshold the blurred movies.  